### 환경변수 로드

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 기본 LLM 호출

In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0,
)
llm.invoke("지구의 자전 주기는?")  # 모델 호출

AIMessage(content='지구의 자전 주기는 **약 23시간 56분 4초**입니다.  하지만 보통 **24시간**으로 간략하게 표현합니다.  24시간과의 차이는 태양에 대한 지구의 공전 때문입니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f70b5ca9-bde1-4ac2-afec-a5b0aceafb7c-0', usage_metadata={'input_tokens': 9, 'output_tokens': 63, 'total_tokens': 72})

### 프롬프트 템플릿 적용

In [4]:
from langchain_core.prompts import ChatPromptTemplate

# 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    "You are an expert in astronomy. Answer the question. <Question>: {input}"
)
print(prompt)

# 템플릿과 LLM을 하나의 체인으로 연결
chain = prompt | llm

# 체인 호출
chain.invoke({"input": "지구의 자전 주기는?"})

input_variables=['input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})]


AIMessage(content='지구의 자전 주기는 정확히 24시간이 아니라는 점을 명심해야 합니다.  24시간은 평균 태양일(mean solar day)이며,  이는 태양이 자오선을 연속해서 통과하는 데 걸리는 시간입니다.  하지만 지구의 자전은 완벽한 등속운동이 아니고, 여러 요인에 의해 영향을 받습니다.\n\n따라서, 좀 더 정확하게 답변하려면 다음과 같은 구분이 필요합니다.\n\n* **평균 태양일 (Mean Solar Day):** 약 24시간 (86,400초).  일상생활에서 사용하는 시간 단위입니다.\n\n* **항성일 (Sidereal Day):** 지구가 자신의 축을 중심으로 한 바퀴 자전하는 데 걸리는 시간.  약 23시간 56분 4초 (86,164초).  이는 지구가 태양 주위를 공전하기 때문에 평균 태양일보다 짧습니다.  천문학적 계산에 사용됩니다.\n\n* **진정한 태양일 (Apparent Solar Day):** 태양이 자오선을 연속해서 통과하는 데 걸리는 시간으로, 지구의 공전 궤도가 타원형이기 때문에 매일 조금씩 달라집니다.  평균 태양일과의 차이는 시간 방정식으로 나타냅니다.\n\n\n질문의 의도가 일상적인 시간 개념이라면 **약 24시간**이라고 답할 수 있지만, 천문학적으로 정확한 답을 원한다면 **항성일 (약 23시간 56분 4초)** 또는 **평균 태양일 (약 24시간)**을 명시하고 그 차이점을 설명해야 합니다.  단순히 "24시간"이라고만 답하는 것은 과학적으로 부정확합니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f5ae2b4d-7aef-48ad-8599-74f2cc8cec76-0', usage_metadata={'input_tokens': 23, 'output_to

### Output parser 적용

In [5]:
from langchain_core.output_parsers import StrOutputParser

# Output parser 정의 - 문자열로 파싱
output_parser = StrOutputParser()

# 프롬프트 템플릿과 LLM, Output parser를 하나의 체인으로 연결
chain = prompt | llm | output_parser

# 체인 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 정확히 24시간이 아니라는 점을 명심해야 합니다.  24시간은 평균 태양일(mean solar day)을 기준으로 한 값이며,  이는 지구가 태양을 중심으로 한 바퀴 도는 데 걸리는 시간을 측정한 것입니다.  하지만 지구의 자전은 완벽한 등속운동이 아니기 때문에,  측정 방법에 따라 자전 주기는 약간씩 달라집니다.\n\n* **항성일(sidereal day):** 지구가 별을 기준으로 한 바퀴 자전하는 데 걸리는 시간입니다.  이는 약 23시간 56분 4초입니다.  지구가 태양 주위를 공전하기 때문에, 태양을 기준으로 한 자전 주기와는 차이가 발생합니다.\n\n* **태양일(solar day):** 지구가 태양을 기준으로 한 바퀴 자전하는 데 걸리는 시간입니다.  이는 평균 태양일(mean solar day)과 진정 태양일(apparent solar day)로 나뉩니다.  평균 태양일은 24시간으로 정의되지만, 진정 태양일은 지구 공전 궤도의 타원형 때문에 약간씩 변동합니다.\n\n따라서 "지구의 자전 주기는?" 이라는 질문에 대한 답은 맥락에 따라 달라집니다.  일반적인 대화에서는 24시간(평균 태양일)이라고 답하는 것이 적절하지만, 천문학적인 정확성을 요구하는 경우에는 항성일(약 23시간 56분 4초)을 명시해야 합니다.  더욱 정확한 값을 원한다면,  지구 자전의 불규칙성을 고려한 더욱 복잡한 계산이 필요합니다.\n'